In [1]:
import numpy as np
from scipy.signal import find_peaks, savgol_filter
from scipy.integrate import quad, solve_ivp
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict

In [2]:
def nested_dict():
    return defaultdict(nested_dict)

In [3]:
SMALL_SIZE = 22
MEDIUM_SIZE = 24
BIGGER_SIZE = 24

plt.rc('font', size=BIGGER_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the figure title

plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['text.usetex'] = True

In [4]:
mu_1 = -0.8 # Magnetic moment of da big magnet
M_2 = 631000 / 2 # Magnetization of da small magnet
M_2_err = 55000 # Small magnet magnetization error
g = 9.807 # Gravitational acceleration
m = 64.78e-3 # Magnet weight
L = 0.1 # Magnet length
D = 0.01 # Magnet diameter
h = 16e-3 # Distance between magnets

In [5]:
def force_two_magnets(r, alpha, beta, m_1, m_2):
    force_magnitude = -3e-7 * (m_1 * m_2) * r**-4
    force_radial = force_magnitude * (2 * np.cos(alpha) * np.cos(alpha - beta) - np.sin(alpha) * np.sin(alpha - beta))
    force_angular = force_magnitude * np.sin(2 * alpha - beta)

    return force_radial, force_angular

def infinitesimal_magn_moment(d):
    return 2 * M_2 * L * np.sqrt(D**2 / 4 - (d - D/2)**2)

def dipole_position(d, theta):
    return [D/2 - d*np.cos(theta), -d*np.sin(theta)]

def magnet_torque(d, theta):
    x_dipole, y_dipole = dipole_position(d, theta)
    dipole_dist = np.hypot(x_dipole, y_dipole-h)
    dipole_angle = np.arctan2(x_dipole, -(y_dipole - h))
    F_r, F_theta = force_two_magnets(dipole_dist, dipole_angle, theta, mu_1, infinitesimal_magn_moment(d))
    torque = (F_r * np.cos(dipole_angle) - F_theta * np.sin(dipole_angle)) * d * np.cos(theta)

    return torque

In [6]:
angles = np.linspace(-np.pi/2, np.pi/2, 100)

torques = np.zeros(100)

for i, theta in enumerate(angles):
    torques[i] = quad(lambda x: magnet_torque(x, np.abs(theta)), 0, D)[0] * np.sign(theta)

In [7]:
prev_theta = 0
restitution = 0.9

def theta_derivatives(t, x):
    global prev_theta
    omega = x[1]
    theta = x[0]

    damping = 0
    if np.sign(theta) != np.sign(prev_theta):
        damping = -0.05 * omega

    prev_theta = theta
    magnet_t = quad(lambda x: magnet_torque(x, abs(theta)), 0, D)[0]
    # magnet_t = 0
    angular_accel = damping + (np.sign(theta * (np.abs(theta) - np.arctan2(D, L))) * (-m * g * (np.hypot(D, L) / 2) * np.sin(np.abs(theta) + np.arctan2(D, L))) - magnet_t * np.sign(theta)) / (m * L**2 / 12 + m * D**2 / 4 + m * (L**2 + D**2) / 4)

    return [omega, angular_accel]

In [8]:
time = np.linspace(0, 2, 10000)

theta_0 = np.radians(50)

sol = solve_ivp(theta_derivatives, [time[0], time[-1]], y0=[theta_0, 0], t_eval=time, method='DOP853')

KeyboardInterrupt: 

In [ ]:
plt.plot(sol.t, np.degrees(sol.y[0]))
plt.plot(sol.t, sol.y[1])

In [ ]:
def euler_euler(theta_0, t_end, dt):
    thetas = np.zeros(int(t_end // dt))
    omegas = np.zeros(int(t_end // dt))

    thetas[0] = theta_0
    theta = theta_0
    omega = 0

    time = 0

    for i in range(1, int(t_end // dt)):
        magnet_t = quad(lambda x: magnet_torque(x, abs(theta)), 0, D)[0]
        angular_accel = (np.sign(theta * (np.abs(theta) - np.arctan2(D, L))) * (-m * g * (np.hypot(D, L) / 2) * np.sin(np.abs(theta) + np.arctan2(D, L))) - magnet_t * np.sign(theta)) / (m * L**2 / 12 + m * D**2 / 4 + m * (L**2 + D**2) / 4)

        omega += angular_accel * dt
        theta += omega * dt

        if np.sign(thetas[i-1]) != np.sign(theta):
            print(i)
            omega *= 0.946

        thetas[i] = theta
        omegas[i] = omega

    return thetas, omegas

In [ ]:
t_end = 2
dt = 0.0001

thetas, omegas = euler_euler(np.radians(50), t_end, dt)

In [ ]:
with open('../data_processing/data.pickle', 'rb') as f:
    data_map = pickle.load(f)
a = data_map[10][100][40]
a['t'] = a['t'] - a['t'].iloc[76]
a['t'] = a['t'][a['t'] > 0]
a['theta'] = savgol_filter(a['theta'], 10, 3)

In [ ]:
a['theta'][a['theta'] < 0] = a['theta'][a['theta'] < 0] * 0.9

In [ ]:
plt.xlabel('Time (s)')
plt.ylabel('Angle ($^\circ$)')
plt.title(r'$\theta_0 \approx 50^\circ$, $L \approx 100$\,mm, $D \approx 10$\,mm')

plt.plot(a['t'][a['t'] < 2], a['theta'][a['t'] < 2] * 180 / np.pi, linestyle='None', marker='o', label='data')
plt.plot(np.arange(0, 2, dt)[:-1], np.degrees(thetas), label='model     ')

plt.legend(loc='upper left', bbox_to_anchor=(1, 0.9))
plt.savefig('oscillations_cor.png', bbox_inches='tight')

In [ ]:
def period(theta_0):
    time = np.linspace(0, 2, 1000)

    sol = solve_ivp(theta_derivatives, [time[0], time[-1]], y0=[theta_0, 0], t_eval=time, method='Radau')

    peaks, _ = find_peaks(sol.y[0])
    return (sol.t[peaks[1]] - sol.t[peaks[0]]), sol.y[0], sol.y[1]

In [ ]:
angles = np.linspace(20, 80, 10)

M_2_arr = [M_2 / 6 - M_2_err, M_2 / 6 + M_2_err]
# M_2_arr = [631000]
periods = [[], []]
thetas_arr = [[], []]
omegas_arr = [[], []]

for i, M_2_a in enumerate(M_2_arr):
    M_2 = M_2_a
    print(M_2)
    for angle in angles:
        T, thetas, omegas = period(np.radians(angle))
        periods[i].append(T)
        thetas_arr[i].append(thetas)
        omegas_arr[i].append(omegas)
        print(angle)

In [ ]:
with open('../data_processing/periods.pickle', 'rb') as f:
    periods_df = pickle.load(f)

In [ ]:
periods[0]

In [ ]:
d = 4
l = 100

plot_data = periods_df[(periods_df['l'] == l) & (periods_df['d'] == d)]
plt.errorbar(
    x=plot_data['initial_angle'],
    y=plot_data['period'],
    yerr=plot_data['period_std'],
    linestyle='None',
    marker='o',
    capsize=4,
    color='darkorange',
    label='experiments'
)

plt.fill_between(angles, periods[0], periods[1], alpha=0.2, label='model')
plt.plot(angles, np.average(periods, axis=0))

plt.legend(
    fancybox=True
)

plt.title("$d \\approx {}\,$mm, $l \\approx {}\,$mm".format(d, l))

plt.xlabel("Initial angle $\\theta_0\\,$($^\circ$)")
plt.ylabel("Period $T$ (s)")

# plt.savefig("period_initial_angle_length{}_d{}_model.png".format(l, d), bbox_inches="tight")

In [ ]:
with open("d_{}_l_{}_model.pickle".format(d, l), "wb") as f:
    pickle.dump(periods, f)

In [ ]:
from scipy.signal import find_peaks

peaks, _ = find_peaks(-np.abs(theta))